In [1]:
import os, sys 
import numpy as np 

IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)

from imatools.common import ioutils as iou
from imatools.common import vtktools as vtku 

def calc_cog(pts, el):
    return [np.mean(pts[ee], 0) for ee in el]

norm2 = lambda a : np.linalg.norm(a)
norm_vec = lambda a : a/norm2(a)


def extract_from_dataframe(dframe, window, bdir=None):
    _dir = dframe.result_path[window]
    _user = dframe.user[window].tolist()
    _patient = dframe.patient[window].tolist()
    _mode = dframe['mode'][window].tolist()
    _original_dir = dframe.original_path[window]
    _sim_dir = dframe.simulation_path[window]

    if (bdir is not None):
        _dir = [px.replace('$AFIB_REPROD', bdir) for px in _dir]
        _original_dir = [px.replace('$AFIB_REPROD', bdir)
                         for px in _original_dir]
        _sim_dir = [px.replace('$AFIB_REPROD', bdir) for px in _sim_dir]
    else:
       _dir = _dir.tolist()
       _sim_dir = _sim_dir.tolist()
       _original_dir = _original_dir.tolist()

    return _dir, _user, _patient, _mode, _original_dir, _sim_dir


In [16]:
import pandas as pd 

# Locations of hard drive based on platform
dirdic = {'macOS': '/Volumes/sandisk',
          'Linux': '/media/jsl19/sandisk',
          'Windows': 'D:/'}

tex_dic = {'macOS' : '/Users/jsolislemus/Documents/TEX', 
            'Linux' : '/home/jsl19/Documents/tex'}

base_dir = iou.fullfile(dirdic[iou.chooseplatform()], '01_atrialfibres/06_Reproducibility/05_UserProjects')
sims_dir = iou.fullfile(base_dir, '008_simulation_results')
out_dir = iou.fullfile(base_dir, '009_simulation_images', 'Fibre_Agreement')

tex_dir = iou.fullfile(tex_dic[iou.chooseplatform()], 'tex.cinbio.reproducibility/scripts/py')

N = ['M' + str(n) for n in np.linspace(1,100,num=100, dtype=int)]


df_tracking = pd.read_csv(iou.fullfile(base_dir, 'simulations_paths.csv'))
num_pairs = int(len(df_tracking)/2)

df_tracking

,ID,original_path,simulation_path,result_path,user,patient,mode,processing
0,M001,$AFIB_REPROD/002_charlie/03_completed/PATIENT_...,$AFIB_REPROD/007_simulations/M001,$AFIB_REPROD/008_simulation_results/M1,002_charlie,PATIENT_1877870,inter,manual
1,M002,$AFIB_REPROD/003_rosie/03_completed/PATIENT_18...,$AFIB_REPROD/007_simulations/M002,$AFIB_REPROD/008_simulation_results/M2,003_rosie,PATIENT_1877870,inter,automatic
2,M003,$AFIB_REPROD/002_charlie/03_completed/PATIENT_...,$AFIB_REPROD/007_simulations/M003,$AFIB_REPROD/008_simulation_results/M3,002_charlie,PATIENT_2540680,inter,automatic
3,M004,$AFIB_REPROD/004_jose/03_completed/PATIENT_254...,$AFIB_REPROD/007_simulations/M004,$AFIB_REPROD/008_simulation_results/M4,004_jose,PATIENT_2540680,inter,automatic
4,M005,$AFIB_REPROD/002_charlie/03_completed/PATIENT_...,$AFIB_REPROD/007_simulations/M005,$AFIB_REPROD/008_simulation_results/M5,002_charlie,PATIENT_3506490,inter,automatic
...,...,...,...,...,...,...,...,...
95,M096,$AFIB_REPROD/004_jose/03_completed/PATIENT_681...,$AFIB_REPROD/007_simulations/M096,$AFIB_REPROD/008_simulation_results/M96,004_jose,PATIENT_6816821,intra,automatic
96,M097,$AFIB_REPROD/004_jose/03_completed/PATIENT_148...,$AFIB_REPROD/007_simulations/M097,$AFIB_REPROD/008_simulation_results/M97,004_jose,PATIENT_1485560,intra,automatic
97,M098,$AFIB_REPROD/004_jose/03_completed/PATIENT_148...,$AFIB_REPROD/007_simulations/M098,$AFIB_REPROD/008_simulation_results/M98,004_jose,PATIENT_1485561,intra,automatic
98,M099,$AFIB_REPROD/004_jose/03_completed/PATIENT_132...,$AFIB_REPROD/007_simulations/M099,$AFIB_REPROD/008_simulation_results/M99,004_jose,PATIENT_1321410,intra,automatic


In [ ]:
import shutil

def copy_to_comparisons_dir(imsh, scar, fib_dir, odir, pre) : 
    """Copies mesh files for easier comparisons 

    imsh_ : Original input mesh name /path/clean-Labelled-refined  
    scar_ : /path/MaxScar_Normalised.vtk
    fib_dir : Fibre files dir
    odir_ : output directory
    pre_  : Prefix (MX)
    """
    exts = ['.pts', '.elem', '.lon']
    dat_files = ['LAT_RSPV_1.dat', 'LAT_RSPV_l.dat', 'PSNode.dat', 'PSNodeSmooth.dat']

    omsh = iou.fullfile(odir, pre)
    os.makedirs(omsh, exist_ok=True)

    shutil.copyfile(src=scar, dst=iou.fullfile(omsh, 'scar.vtk'))
    shutil.copyfile(src=imsh+'.vtk', dst=iou.fullfile(omsh, 'input.vtk'))

    for ix in range(len(exts)) : 
        shutil.copyfile(src=imsh+exts[ix], dst=iou.fullfile(omsh, 'input'+exts[ix]))

        shutil.copyfile(src=iou.fullfile(fib_dir, 'Bilayer_1'+exts[ix]), dst=iou.fullfile(omsh, 'fibre_1'+exts[ix]))
        shutil.copyfile(src=iou.fullfile(fib_dir, 'Bilayer_l'+exts[ix]), dst=iou.fullfile(omsh, 'fibre_l'+exts[ix]))
        if (exts[ix] != '.lon') : 
            shutil.copyfile(src=iou.fullfile(fib_dir, 'Monolayer'+exts[ix]), dst=iou.fullfile(omsh, 'mono'+exts[ix]))

    for d in dat_files : 
        shutil.copyfile(src=iou.fullfile(fib_dir, d), dst=iou.fullfile(omsh, d))

def get_names_to_copy(indx, og_dir, sim_dir) : 
    imsh = iou.searchFileByType(og_dir[indx], 'clean', 'vtk')
    imsh = imsh[0][0:-4] # /path/clean-Labelled-reg-refined (no extension)

    scar = iou.fullfile(og_dir[indx], 'MaxScar_Normalised.vtk')
    fib_dir = sim_dir[indx] 

    return imsh, scar, fib_dir 

    
num_comparisons = 50 
N = ['M' + str(n) for n in np.linspace(1,100,num=100, dtype=int)]

pairs = np.arange(100).reshape((num_comparisons, 2))
sim_res_dir, users, patients, mode, original_dir, _ = extract_from_dataframe(df_tracking, window=np.arange(100), bdir=base_dir)

midic_comp_info = []
for which_pair in range(num_comparisons) : 
    comparison_dir = iou.fullfile(base_dir, '011_comparisons', 'C'+str(which_pair))
     
    ix0 = pairs[which_pair, 0]
    imsh0, scar0, fib_dir0 = get_names_to_copy(indx=ix0, og_dir=original_dir, sim_dir=sim_res_dir)
    # copy_to_comparisons_dir(imsh=imsh0, scar=scar0, fib_dir=fib_dir0, odir=comparison_dir, pre=N[ix0])
    
    midic_comp_info.append(
        {
            'original_path' : original_dir[ix0], 
            'result_path' : sim_res_dir[ix0], 
            'comparison_path' : iou.fullfile(comparison_dir, N[ix0]),
            'mode' : mode[ix0], 
            'processing' : df_tracking.iloc[ix0].processing 
        }
    ) 

    ix1 = pairs[which_pair, 1]
    imsh1, scar1, fib_dir1 = get_names_to_copy(indx=ix1, og_dir=original_dir, sim_dir=sim_res_dir)
    # copy_to_comparisons_dir(imsh=imsh1, scar=scar1, fib_dir=fib_dir1, odir=comparison_dir, pre=N[ix1])

    midic_comp_info.append(
        {
            'original_path' : original_dir[ix1], 
            'result_path' : sim_res_dir[ix1], 
            'comparison_path' : iou.fullfile(comparison_dir, N[ix1]),
            'mode' : mode[ix1], 
            'processing' : df_tracking.iloc[ix1].processing 
        }
    ) 

comparisons_ofile = iou.fullfile(base_dir, '011_comparisons', 'comparisons_path.csv')
df_comparisons = pd.DataFrame(midic_comp_info)
df_comparisons.to_csv(comparisons_ofile, index=False)

Load original meshes, calculate Hausdorff filter and save to file 


In [8]:
import vtk

num_comparisons = 50 
N = ['M' + str(n) for n in np.linspace(1,100,num=100, dtype=int)]

# pairs = np.arange(100).reshape((num_comparisons, 2))
# sim_res_dir, users, patients, _, original_dir, _ = extract_from_dataframe(df, window=np.arange(100), bdir=base_dir)

comparison_dir = [iou.fullfile(base_dir, '011_comparisons', 'C'+str(c)) for c in np.arange(num_comparisons)]

cx=0

this_comparison = comparison_dir[cx]
sub_dirs = os.listdir(this_comparison)
names = {'scar' : 'scar', 'l' : 'fibre_l', '1' : 'fibre_1', 'in' : 'input'}

which_name = 'in' # in, scar, l, 1
mname_ext = names[which_name] + '.vtk'
type_of_mapping = 'elem' # elem, pts

id_left = sub_dirs[0]
id_right = sub_dirs[1]
path_left = iou.fullfile(this_comparison, id_left, mname_ext)
path_right = iou.fullfile(this_comparison, id_right, mname_ext)

# midic = vtku.create_mapping(msh_left_name=path_left, msh_right_name=path_right, left_id=id_left, right_id=id_right, map_type=type_of_mapping)


In [22]:

df_comparisons = pd.read_csv(iou.fullfile(base_dir, '011_comparisons', 'comparisons_path.csv'))
cpaths = df_comparisons.comparison_path.tolist()

cpaths = [c.replace('/Volumes', '/media/jsl19') for c in cpaths]

for cp in cpaths :
    ff = iou.fullfile(cp, 'fibre_l')
    os.system('meshtool convert -imsh={} -ifmt=carp_txt -omsh={} -ofmt=vtk_polydata -scale=1e-3'.format(ff, ff))

    lat=iou.fullfile(cp, 'LAT_RSPV_l.dat')
    olat=iou.fullfile(cp, 'lat_l')
    os.system('meshtool extract gradient -msh={} -ifmt=carp_txt -idat={} -odat={}'.format(ff, lat, olat))

OpenMP thread utilization: 20 / 20 threads.
Reading mesh: /media/jsl19/sandisk/01_atrialfibres/06_Reproducibility/05_UserProjects/011_comparisons/C0/M1/fibre_1
Reading elements in text CARP format:                  [============] 00h:00m:00s ETA 
Reading points in text CARP format:                    [============] 00h:00m:00s ETA 
Reading fibers (1 directions) in text CARP format:     [============] 00h:00m:00s ETA 
Done in 1.91131 sec
Computing connectivity graphs:                         [============] 00h:00m:00s ETA 
Writing full mesh connectivity to: /media/jsl19/sandisk/01_atrialfibres/06_Reproducibility/05_UserProjects/011_comparisons/C0/M1/fibre_1.fcon
Output data is node based.
Reading /media/jsl19/sandisk/01_atrialfibres/06_Reproducibility/05_UserProjects/011_comparisons/C0/M1/LAT_RSPV_1.dat: [============] 00h:00m:00s ETA 
Input data is node based.
Writing /media/jsl19/sandisk/01_atrialfibres/06_Reproducibility/05_UserProjects/011_comparisons/C0/M1/lat_1.grad.vec: [========

Development of technique below 

In [ ]:
import vtk

num_comparisons = 50 
N = ['M' + str(n) for n in np.linspace(1,100,num=100, dtype=int)]

# pairs = np.arange(100).reshape((num_comparisons, 2))
# sim_res_dir, users, patients, _, original_dir, _ = extract_from_dataframe(df, window=np.arange(100), bdir=base_dir)

comparison_dir = [iou.fullfile(base_dir, '011_comparisons', 'C'+str(c)) for c in np.arange(num_comparisons)]

cx=0

this_comparison = comparison_dir[cx]
sub_dirs = os.listdir(this_comparison)
names = {'scar' : 'scar', 'l' : 'fibre_l', '1' : 'fibre_1', 'in' : 'input'}

# when input is only vtk (scar)
path_left = iou.fullfile(this_comparison, sub_dirs[0])
path_right = iou.fullfile(this_comparison, sub_dirs[1])

which_name = 'in' # in, scar, l, 1
mname_ext = names[which_name] + '.vtk'
type_of_mapping = 'elem' # elem, pts

msh_left = vtku.readVtk(iou.fullfile(path_left, mname_ext))
msh_right = vtku.readVtk(iou.fullfile(path_right, mname_ext))

# cells 
tot_left = msh_left.GetNumberOfCells()
tot_right = msh_right.GetNumberOfCells() 

path_large = path_left  # 0
path_small = path_right # 1
tot_large = tot_left
tot_small = tot_right
sdir_large = sub_dirs[0]
sdir_small = sub_dirs[1]

if tot_left < tot_right : 
    path_large = path_right
    path_small = path_left
    tot_large = tot_right
    tot_small = tot_left
    sdir_large = sub_dirs[1]
    sdir_small = sub_dirs[0]

msh_small = vtku.readVtk(iou.fullfile(path_small, mname_ext)) # 1
cog_small = vtku.getCentreOfGravity(msh_small)

msh_large = vtku.readVtk(iou.fullfile(path_large, mname_ext)) # 0

cell_locate_on_large=vtk.vtkCellLocator()
cell_locate_on_large.SetDataSet(msh_large)
cell_locate_on_large.BuildLocator()

cell_ids_small = np.arange(tot_small)
cell_ids_large = np.zeros(tot_small, dtype=int)
l2_norm_manual = np.zeros(tot_small, dtype=float)
l2_norm_filter = np.zeros(tot_small, dtype=float)
x_cog_small = cog_small[:, 0]
y_cog_small = cog_small[:, 1]
z_cog_small = cog_small[:, 2]
x_in_large = np.zeros(tot_small, dtype=float)
y_in_large = np.zeros(tot_small, dtype=float)
z_in_large = np.zeros(tot_small, dtype=float)

for ix in range(tot_small): # tot_small
    cellId_in_large = vtk.reference(0)
    c_in_large = [0.0, 0.0, 0.0]
    subId = vtk.reference(0)
    dist_to_large = vtk.reference(0.0)

    cell_locate_on_large.FindClosestPoint(cog_small[ix], c_in_large, cellId_in_large, vtk.reference(0), dist_to_large)
    cell_ids_large[ix] = cellId_in_large.get()

    l2_norm_manual[ix] = np.linalg.norm(cog_small[ix]-c_in_large)
    l2_norm_filter[ix] = dist_to_large
    
    x_in_large[ix] = c_in_large[0]
    y_in_large[ix] = c_in_large[1]
    z_in_large[ix] = c_in_large[2]

midic = {
    sdir_small : cell_ids_small, 
    sdir_large : cell_ids_large, 
    'distance_manual' : l2_norm_manual, 
    'distance_auto'  : l2_norm_filter, 
    'X_'+sdir_small.lower() : x_cog_small, 
    'Y_'+sdir_small.lower() : y_cog_small, 
    'Z_'+sdir_small.lower() : z_cog_small, 
    'X_'+sdir_large.lower() : x_in_large,
    'Y_'+sdir_large.lower() : y_in_large,
    'Z_'+sdir_large.lower() : z_in_large
}

df = pd.DataFrame(midic)
out_dir = iou.fullfile(this_comparison, 'MAPPING')
os.makedirs(out_dir, exist_ok=True)
df.to_csv(iou.fullfile(out_dir, names[which_name] + '_' + type_of_mapping + '.csv'), index=False)


